# Required Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.optimize import dual_annealing
from openpyxl import load_workbook


# Required Functions 

In [6]:
%run Optimization_RTK_Functions.ipynb

# Input

In [9]:
file_path = './CCW_event_9.xlsx'                # Sets the file path for the Excel data file.
data = pd.read_excel(file_path, skiprows=0)     # Reads the data from the Excel file into a DataFrame.

rainfall = data.iloc[:,2].dropna().tolist()     # Gets data from the 3rd column (index 2) for rainfall, drops empty rows, and converts to a list.
obs_rdii = data.iloc[:,1].tolist()              # Gets data from the 2nd column (index 1) for observed RDII flow and converts to a list.

delta_t = 600                                   # Sets the time step duration to 600 seconds (which is a 10-minute interval).
area_acres = 491.153                            # Defines the catchment area in acres.

total_rdii_period = (len(obs_rdii) - len(rainfall)) * delta_t # Calculates the decay period after rainfall stops.

### T and K Range 

In [13]:
# Allowed values for T (in seconds, 10-minute steps)
allowed_T1_values = [x * 60 for x in range(10, 121, 10)]  # 10 min to 2 hrs
allowed_T2_values = [x * 60 for x in range(20, 241, 10)]  # 20 min to 4 hrs
allowed_T3_values = [x * 60 for x in range(30, 421, 10)]  # 30 min to 7 hrs

# Allowed values for K (0.001 steps)
allowed_K1_values = [round(x, 3) for x in np.arange(1, 2.01, 0.001)]  # 1to 2
allowed_K2_values = [round(x, 3) for x in np.arange(2, 3.01, 0.001)]  # 2 to 3
allowed_K3_values = [round(x, 3) for x in np.arange(3.0, 7.01, 0.001)]  # 3 to 7


# Simulated Annealing

## Objective Function defination 

In [ ]:
def map_index_to_value(index, allowed_values):
    """Map a continuous index to the closest value in the allowed set."""
    index = int(round(index))  # Ensure it's an integer
    index = max(0, min(index, len(allowed_values) - 1))  # Clip to valid range
    return allowed_values[index]

def objective_function(params_flat):
    # Unpack parameters (R remains continuous, T and K are indices)
    R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx = params_flat

       # **Strict Non-Negative Constraint for R Values**
    if R1 < 0 or R2 < 0 or R3 < 0:
        print("Invalid R values detected (negative), skipping iteration.")
        return float('inf')

    # Map indices to actual discrete values
    T1 = map_index_to_value(T1_idx, allowed_T1_values)
    T2 = map_index_to_value(T2_idx, allowed_T2_values)
    T3 = map_index_to_value(T3_idx, allowed_T3_values)

    K1 = map_index_to_value(K1_idx, allowed_K1_values)
    K2 = map_index_to_value(K2_idx, allowed_K2_values)
    K3 = map_index_to_value(K3_idx, allowed_K3_values)
    
    # Parameters are now R (continuous) and T/K (discrete via mapping)
    params = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]

    # Proceed with sim_rdii, RMSE, and penalties as before
    sim_rdii = RDII_calculation(params, delta_t, rainfall, area_acres)

    # if not sim_rdii or len(sim_rdii) == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
    #     return float('inf')
    # if len(sim_rdii) != len(obs_rdii):
    #     sim_rdii = sim_rdii[:len(obs_rdii)]

    #     sim_rdii = np.array(sim_rdii)

    # Check for invalid outputs
    sim_rdii = np.array(sim_rdii)
    
    if sim_rdii.size == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
        print("Invalid sim_rdii generated, returning large penalty.")
        return float('inf')

    # Pad the shorter array with zeros to match the length of the longer one
    max_length = max(len(obs_rdii), len(sim_rdii))
    obs_rdii_padded = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
    sim_rdii_padded = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')


    
    # Calculate fitness value
    fitness_value = fitness(
        obs_rdii_padded,
        sim_rdii_padded,
        delta_t,
        weight_rmse=0.25,
        weight_r2=0.25,
        weight_pbias=0.25,
        weight_nse=0.25
    )


    # rmse = fitness(obs_rdii, sim_rdii, delta_t)
    # Calculate penalties

    penalty = 0
    Ro = R_calc(rainfall, obs_rdii, delta_t, area_acres)
    
    # Enforce equality: R1 + R2 + R3 = Ro
    r_sum = R1 + R2 + R3
    penalty += 1e6 * (r_sum - Ro)**2  # quadratic penalty would be (r_sum - Ro)**2 if preferred
    
    # Temporal ordering constraints
    if not (T1 < T2 < T3):
        penalty += 1000
    if not (T1 + T1 * K1 < T2 + T2 * K2 < T3 + T3 * K3 <= total_rdii_period):
        penalty += 1000
  


    return fitness_value + penalty


## Multiple Run SA

In [ ]:
# Number of runs
n_runs = 20

# Bounds for the optimization (ensure they align with your parameters)
bounds = [
    (0, 1),  # R1 bounds
    (0, len(allowed_T1_values) - 1),  # T1 index bounds
    (0, len(allowed_K1_values) - 1),  # K1 index bounds
    (0, 1),  # R2 bounds
    (0, len(allowed_T2_values) - 1),  # T2 index bounds
    (0, len(allowed_K2_values) - 1),  # K2 index bounds
    (0, 1),  # R3 bounds
    (0, len(allowed_T3_values) - 1),  # T3 index bounds
    (0, len(allowed_K3_values) - 1)   # K3 index bounds
]

# Lists to store results
all_params = []
all_fitness_scores = []

for run in range(n_runs):
    print(f"Running Simulated Annealing: Run {run + 1}/{n_runs}")
    
    # Perform Simulated Annealing
    result = dual_annealing(
        objective_function,
        bounds=bounds,
        maxiter=1000,  # Maximum number of iterations
        initial_temp=5230.0,  # Initial temperature (optional tuning)
        restart_temp_ratio=2e-5  # Temperature restart ratio
    )
    
    # Extract the best solution
    best_params = result.x
    best_fitness_value = result.fun

    # Extract parameter indices and map to actual T and K values
    R1, T1_index, K1_index, R2, T2_index, K2_index, R3, T3_index, K3_index = best_params

    T1 = allowed_T1_values[int(T1_index)]
    T2 = allowed_T2_values[int(T2_index)]
    T3 = allowed_T3_values[int(T3_index)]

    K1 = allowed_K1_values[int(K1_index)]
    K2 = allowed_K2_values[int(K2_index)]
    K3 = allowed_K3_values[int(K3_index)]

    # Final best parameter set for this run
    best_params_actual = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]
    
    # Store results
    all_params.append(best_params_actual)
    all_fitness_scores.append(best_fitness_value)

print("Completed all runs.")


In [ ]:
# Create a DataFrame with results
columns = ['Run', 'R1', 'T1', 'K1', 'R2', 'T2', 'K2', 'R3', 'T3', 'K3', 'Fitness']
results_data = []

for run, (params, fitness) in enumerate(zip(all_params, all_fitness_scores), start=1):
    results_data.append([
        run,
        params[0][0], params[0][1], params[0][2],  # R1, T1, K1
        params[1][0], params[1][1], params[1][2],  # R2, T2, K2
        params[2][0], params[2][1], params[2][2],  # R3, T3, K3
        fitness  # Fitness value
    ])

results_df = pd.DataFrame(results_data, columns=columns)

# Display results
print("\nResults from all runs:")
print(results_df)

# Save results to a CSV file if needed
#results_df.to_excel("simulated_annealing_results_event5.xlsx", index=False)

# Calculate and display mean and standard deviation for parameters and fitness
mean_values = results_df.mean()
std_values = results_df.std()

print("\nMean values of parameters and fitness across runs:")
print(mean_values)

print("\nStandard deviation of parameters and fitness across runs:")
print(std_values)

## Saving RTK parameters

In [ ]:
# Define the Excel file name
excel_filename = "RTK_Parameters_all_algorithms_Ro_constraint_E9_CCW.xlsx"
sheet_name = "SA"

# Check if "Run" column exists, and add it only if not present
if "Run" not in results_df.columns:
    results_df.insert(0, "Run", range(1, len(results_df) + 1))

# Select the desired columns
export_columns = ["Run", "R1", "T1", "K1", "R2", "T2", "K2", "R3", "T3", "K3"]

# Check if the Excel file exists
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file does not exist, create a new one
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Results successfully saved to {excel_filename} in sheet {sheet_name}.")

In [ ]:
plot=RDII_combined_plot(all_params, delta_t, rainfall, area_acres, obs_rdii)

## Saving Metrics 

In [ ]:
criteria= calculate_criteria_multiple_runs(all_params, obs_rdii, delta_t, rainfall, area_acres )

In [ ]:
# Define the Excel file and sheet name
excel_filename = "Metrices_all_algorithms_Ro_E9_CCW.xlsx"
criteria_sheet_name = "SA"

# Select only the first 20 rows
criteria_df = criteria.iloc[:20].copy()

# Ensure "Run" column is present in criteria_df
if "Run" not in criteria_df.columns:
    criteria_df.insert(0, "Run", range(1, len(criteria_df) + 1))

# Add the "Fitness" column from results_df (first 20 rows)
criteria_df["Fitness"] = results_df.loc[:19, "Fitness"].values

# Select the required columns
export_columns_criteria = ["Run", "RMSE", "R2", "PBIAS", "NSE", "Fitness"]

# Check if the Excel file exists and append or create a new file
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)
except FileNotFoundError:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)

print(f"Criteria successfully saved to {excel_filename} in sheet {criteria_sheet_name}.")